In [ ]:
library(Seurat) #seurat V5
#推荐使用seurat V4
library(dplyr)

In [3]:
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
# options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")

sequential:
- args: function (..., envir = parent.frame(), workers = "<NULL>")
- tweaked: FALSE
- call: plan(sequential)
FutureBackend to be launched

multisession:
- args: function (..., workers = 40)
- tweaked: TRUE
- call: plan("multisession", workers = 40)
MultisessionFutureBackend:
Inherits: ClusterFutureBackend, MultiprocessFutureBackend, FutureBackend
UUID: a7602c83d02f2a1a072664bbac781f38
Number of workers: 40
Number of free workers: 40
Available cores: 80
Automatic garbage collection: FALSE
Early signaling: FALSE
Interrupts are enabled: TRUE
Maximum total size of globals: +Inf
Maximum total size of value: +Inf
Argument ‘rscript_libs’: "/home/guoliming/micromamba/envs/R4.4_Seurat/lib/R/library"
Argument ‘persistent’: FALSE
Argument ‘wait.timeout’: 86400
Argument ‘wait.interval’: 0.01
Argument ‘wait.alpha’: 1.01
Argument ‘hooks’: FALSE
Number of active futures: 0
Number of futures since start: 0 (0 created, 0 launched, 0 finished)
Total runtime of futures: 0 secs (NaN secs/finished future)
Workers of type RichSOCKcluster:
- Summary: Socket cluster with 40 nodes on host ‘localhost’ (R version 4.4.3 (2025-02-28), platform x86_64-conda-linux-gnu)
- [ OK ] Node 1/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56599 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #5 ('<-localhost:11682')]
- [ OK ] Node 2/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56558 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #6 ('<-localhost:11682')]
- [ OK ] Node 3/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56586 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #7 ('<-localhost:11682')]
- [ OK ] Node 4/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56578 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #8 ('<-localhost:11682')]
- [ OK ] Node 5/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56564 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #9 ('<-localhost:11682')]
- [ OK ] Node 6/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56567 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #10 ('<-localhost:11682')]
- [ OK ] Node 7/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56584 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #11 ('<-localhost:11682')]
- [ OK ] Node 8/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56563 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #12 ('<-localhost:11682')]
- [ OK ] Node 9/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56560 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #13 ('<-localhost:11682')]
- [ OK ] Node 10/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56574 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #14 ('<-localhost:11682')]
- [ OK ] Node 11/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56569 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #15 ('<-localhost:11682')]
- [ OK ] Node 12/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56568 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #16 ('<-localhost:11682')]
- [ OK ] Node 13/40: valid connection, alive [RichSOCKnode of a socket cluster on local host 'localhost' with pid 56557 (R version 4.4.3 (2025-02-28), x86_64-conda-linux-gnu) using socket connection #17 ('<-localhost:11682')]
- [ OK ] Node 14/40: valid connection, alive [RichSOCKnode

## 1.1 lappy循环依次读入数据

In [4]:
## 1.1 lappy循环依次读入数据
# Create each individual Seurat object for every sample
fileID = list.files("./1.10X_files/matrix/")
path = "./1.10X_files/matrix/"
fileID

[1] "GSM8504078" "GSM8504079" "GSM8504080" "GSM8504081" "GSM8504082"
[6] "GSM8504083" "GSM8863170" "GSM8863172"

In [5]:
# 运行lapply函数读入数据
seurat.list = lapply(fileID, function(file){
  seurat_data <- Read10X(data.dir = paste0(path, file))
  seurat_obj <- CreateSeuratObject(counts = seurat_data,
                                   min.cells = 0,
                                   min.features = 0,
                                   project = file)
  return(seurat_obj)
})

In [ ]:
names(seurat.list) = fileID
seurat.list

## 1.2 merge多个seurat对象

In [ ]:
## 1.2 merge多个seurat对象
# Create a merged Seurat object
merged_seurat <- merge(x = seurat.list[[1]],
                       y = seurat.list[-1],
                       add.cell.id = fileID)
merged_seurat 

## 1.3 表型数据载入

In [10]:
## 1.3 表型数据载入
merged_seurat$sampleID = merged_seurat$orig.ident

# 根据样本信息重命名编组
merged_seurat$group <- recode(merged_seurat$sampleID,
                              "GSM8504078" = "CRL",
                              "GSM8504079" = "CRL",
                              "GSM8504080" = "CRL",
                              "GSM8863170" = "CRL",
                              "GSM8504081" = "ALI",
                              "GSM8504082" = "ALI",
                              "GSM8504083" = "ALI",
                              "GSM8863172" = "ALI"
                              )

# 'GSM8504078''GSM8504079''GSM8504080''GSM8504081''GSM8504082''GSM8504083''GSM8863170''GSM8863172'

In [11]:
head(merged_seurat@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,sampleID,group
,<chr>,<dbl>,<int>,<chr>,<chr>
GSM8504078_AAACCCAAGATTGAGT-1,GSM8504078,61497,5735,GSM8504078,CRL
GSM8504078_AAACCCAAGCAACAGC-1,GSM8504078,3146,1076,GSM8504078,CRL
GSM8504078_AAACCCACATGTTTGG-1,GSM8504078,546,338,GSM8504078,CRL
GSM8504078_AAACCCAGTAGATCCT-1,GSM8504078,2981,1158,GSM8504078,CRL
GSM8504078_AAACCCAGTATCGTGT-1,GSM8504078,1637,744,GSM8504078,CRL
GSM8504078_AAACCCAGTGAGCGAT-1,GSM8504078,5476,1910,GSM8504078,CRL


## 1.4 保存一下

#### （1）数据保存

In [12]:
#qs速度快
#install.packages('qs')
library(qs)
system.time({
    qsave(merged_seurat,file = "./Outdata/Step1.RawCount_merged_seurat.qs") 
})

   user  system elapsed 
 22.425   3.674  26.593 

#### （2）数据读取

In [13]:
# qread速度很快
library(qs)
system.time({
    input.data = qread(file = "./Outdata/Step1.RawCount_merged_seurat.qs")
           })

   user  system elapsed 
  7.400   3.760  11.226 

In [ ]:
### End